In [1]:
# importing libraries
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# setting working directory and reading database
wd = os.getcwd()

discography_df = pd.read_csv(wd + "/discografia_enredos.csv")

In [3]:
# creating function to analyze song suggestions and find pattern that
# facilitates scraping
def get_input_suggestions(driver_path: str, url: str, search_bar_id: str, 
                            song_name: str, suggestion_list_class: str, headless = False):
    """Given a lyrics site url, searches for the song and
    returns the list of suggestions given by the input search bar"""

    if not headless:
        driver = webdriver.Chrome(driver_path)
    else:
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        driver = webdriver.Chrome(driver_path, chrome_options = options)
    
    # TODO: add handling for timeout and crashing excpections?
    driver.get(url)
    
    search_bar_input = driver.find_element(By.ID, search_bar_id)

    search_bar_input.send_keys(song_name)

    song_suggestions = WebDriverWait(driver, 60).until(
        EC.visibility_of_element_located((By.CLASS_NAME, suggestion_list_class))
    ).get_attribute("innerHTML")

    driver.close()

    return song_suggestions

In [7]:
# TODO: consider pool thread executor, which may imply altering the function and further
# modularize it into separate components (this + headless may greatly improve execution time)
discography_df.loc[:, "suggestions_returned"] = discography_df.loc[:, "enredo"].apply(
    lambda x: get_input_suggestions(
        driver_path = "/Users/bi005521/Desktop/chromedriver", url = "https://www.letras.mus.br/",
        search_bar_id = "main_suggest", song_name = x.replace("\"", ""), suggestion_list_class = "suggest__list",
        headless = True
    ) if isinstance(x, str) else None
)

/var/folders/3n/tvjft5y125xgcl5db_89d6sm0000gt/T/ipykernel_23447/3723222966.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options = options)
/var/folders/3n/tvjft5y125xgcl5db_89d6sm0000gt/T/ipykernel_23447/3723222966.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options = options)
